In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [7]:
# health
obj = Liputan6()
end_date = date(2018, 8, 8)
start_date = date(2018, 2, 6)
init_page = 9
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[0:]

In [ ]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-02-06
news
page  1
https://www.liputan6.com/news/indeks/2018/02/06?page=1
Insert berita  Kuasa Hukum SBY: Janggal Kaitkan Ibas di Proyek E-KTP
masuk
Insert berita  Zumi Zola Tersangka, PAN: Dia Datang dari 3 Generasi yang Baik
masuk
Insert berita  Banjir Jakarta, Warga Kebon Pala Terkena Penyakit Hipotermia
masuk
Insert berita  Kapolri: Ukraina dan Indonesia Punya Toleransi Beragama yang Sama
masuk
Insert berita  Normalisasi Sungai, Sandi: Bukan Digusur, Warga Pindah Sendiri
masuk
Insert berita  SBY Serahkan Bukti Pencemaran Nama Baik oleh Pengacara Setnov
masuk
Insert berita  Novel Baswedan akan Dioperasi Dokter dari Inggris
masuk
Insert berita  Dilaporkan SBY, Firman: Ini Proses Pidana, Jangan Diintimidasi
masuk
Insert berita  Ini Cara Menteri PUPR Atasi Longsor di Puncak
masuk
Insert berita  Sering Difitnah, SBY: Pak Jokowi Suatu Saat Bisa seperti Saya
masuk
Insert berita  Tanggapan Setya Novanto soal Laporan SBY ke Bareskrim
masuk
Insert berita  4 Korban Longsor Cijeruk Ditemu

masuk
Insert berita  Longsor di Perimeter Selatan, Ini Akses ke Bandara Soetta
masuk
Insert berita  KemenPUPR Siagakan Alat Berat di 2 Lokasi Longsor Puncak
masuk
Insert berita  Tahan Laju Air, Polisi Membuat Tanggul Darurat di Puncak Pas
masuk
Insert berita  Sumatera Barat Jadi Tuan Rumah Hari Pers Nasional 2018
masuk
Insert berita  Anies: Puncak Banjir Kiriman Sudah Lewat
masuk
Insert berita  Arumi Bachsin Resmi Jadi Kader PAN
masuk
Insert berita  Pasutri Perampok Sadis di Cimahi Pernah Bunuh Korbannya
masuk
Insert berita  Data Nasabah Kartu Kredit Diintip
masuk
Insert berita  Menteri LHK Ungkap Penyebab Longsor di Puncak dan Bogor
masuk
page  8
https://www.liputan6.com/news/indeks/2018/02/06?page=8
Insert berita  Jokowi Akan Hadiri Peringatan Hari Pers Nasional 2018
masuk
Insert berita  Polisi Kerahkan 300 Personel Cari Korban 2 Titik Longsor di Bogor
masuk
Insert berita  Operator Berpotensi Jadi Tersangka Kasus Crane Jatuh Jatinegara
masuk
Insert berita  Derasnya Aliran Ciliwung Me